# Demonstration of anticipated end-to-end creation of Orbit PreCalculations
 - This is the expected usage to go from a newly-fitted orbit, to the storage of cheby polynomials & nightly-healpix in a database
 
 - At present (20200618) this is incomplete and intended as a schematic to aid development

#### Basic imports ...

In [1]:
import time
import numpy as np
import scipy.stats as stats
import math
import random
from collections import defaultdict
import os
import sys
from collections import Counter
import glob 
import warnings
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import importlib
from astropy.time import Time
import pickle


# -------------------------------------------------------------------------------------
# Local imports
# -------------------------------------------------------------------------------------

# orbit_cheby & nbody_reader imports 
#sys.path.append( "/Users/matthewjohnpayne/Envs/orbit_cheby/orbit_cheby/" )
from orbit_cheby import orbit_cheby
from orbit_cheby import nbody_reader
importlib.reload(orbit_cheby)
importlib.reload(nbody_reader)

# obs_pos import(s) 
from orbit_cheby import obs_pos
importlib.reload(obs_pos)

# sql import(s) 
from orbit_cheby import sql
importlib.reload(sql)

# PreCalc import(s) 
from orbit_cheby import precalc
importlib.reload(precalc)


ModuleNotFoundError: No module named 'orbit_cheby'

### (1) Use *mpc_nbody*
 - Perform NBody simulation on results from OrbFit
 - mpc_nbody is a convenience wrapper around Holman's version of reboundx

In [ ]:
%%time
importlib.reload(mpc_nbody)
# First, let's initiate the class with an input file:
Sim = mpc_nbody.NbodySim(filenames[0], 'eq')

# Now run the integrator, by calling the object. 
Sim(tstep=20, trange=600)

#The results are all available inside the object now:
for item in Sim.__dict__ : print(item)

### (2) Use *orbit_cheby*
 - Load arrays of simulation data into Multi-Sector-Cheby object(s)

In [ ]:
# Use the MSC_Loader to do all of the work to decalre and populate a list of MSC objects
#
# NEEDS TO BE UPDATED TO USE Sim.output_times & Sim.output_vectors
#
MSCs = orbit_cheby.MSC_Loader(FROM_ARRAY = True , 
                                primary_unpacked_provisional_designations = name, 
                                times_TDB = times, 
                                statearray = states).MSCs

### (3) Use *orbit_cheby*
 - Uses the chebyshev coeffs in the MSCs to calculate nightly healpix
 - Stores the chebyshev coeffs & the nightly healpix in a database

In [ ]:
# Declare a "PreCalc" object 
P = precalc.PreCalc()

# Do the upsert 
P.upsert( MSCs , observatoryXYZ)